## 调度系统

在360内部我们使用[XLearning](https://github.com/Qihoo360/XLearning)完成TensorNet程序的提交和调度。不过TensorNet不局限于使用XLearning调度，任何可以管理MPI任务的调度系统都可以。

## 作业提交

提交作业的时候除了将自己模型的Python文件提交到集群外，还需要将依赖的包一并打包上去。

In [ ]:
$ FILES="tensornet.tar.gz,wide-deep.py,job_main.sh"
$ MPI_QUEUE="xxxx"
$ CMD="sh job_main.sh"

**job_main.sh**的内容如下

In [ ]:
$ cat job_main.sh

# 解压tensornet包，准备环境
function prepare_env() {
    tar -zxvf lib.tar.gz > /dev/null

    (
        mkdir -p tn
        cd tn
        mv ../tensornet.tar.gz .
        tar -zxvf tensornet.tar.gz > /dev/null
    )

    return 0
}

function start_job()
{
    export PYTHONPATH="$(pwd)/tn:${PYTHONPATH}"
    export LD_LIBRARY_PATH="$(pwd)/lib:${JAVA_HOME}/jre/lib/amd64/server:${HADOOP_HOME}/lib/native"

    (
        set -o xtrace

        ${PYTHON3} wide-deep.py                         \
            --data_dir="${YARN_HDFS}${TF_RECORD_PATH}"  \
            --match_pattern='part*'
    )

    return 0
}

prepare_env
start_job

执行提交运行。

In [ ]:
$ xl-submit \
   --app-type "tensornet"                \
   --app-name "wide-deep"                \
   --worker-num 50                       \
   --worker-memory 50G                   \
   --worker-cores 6                      \
   --files "${FILES}"                    \
   --queue "${MPI_QUEUE}"                \
   --launch-cmd "${CMD}"

## NOTICE

在我们使用XLearning的时候没有使用它可以指定`ps`节点的功能，只需要指定`worker`即可，如果使用其它调度工具应该不需要操心这个。